# AILab 画像分類コンペティション

## ルール
検証にCIFAR10のValデータは使わないでください。

(今回評価用のデータとして用いるため)

モデルを評価する際は、Trainデータを分割して使ってください。

In [ ]:
#ライブラリのインポート
import torch as torch
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm
from torchsummary import summary
from pylab import rcParams

from utils import make_dataset
from utils import eval

### 1.1. データセットの読み込み（改変不可）
- pytorchではデータをTensor形式で扱います。(listやnumpyと似たような構造を持つ)

In [ ]:
#このセルは改変しないで
train_dataset=make_dataset()

### 1.2. データセットの可視化

In [ ]:
%matplotlib inline
def show(img):
    npimg = img.numpy() 
    plt.grid(False) 
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

print (train_dataset)
image, label = train_dataset[1]
print("学習画像サイズ",image.size())
print ("ラベル:",label)
show(image)

### 1.3. データセットをミニバッチ単位に変換します。
- **torch.utils.data.DataLoader**クラスで行います。

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, #バッチサイズの定義
                                           shuffle=True,
                                          num_workers=1)



事前学習あり

In [ ]:
from torchvision import models
class VGG(nn.Module):
  def __init__(self):
    super(VGG,self).__init__()
    n_units = 2048*2
    vgg = models.vgg19_bn(pretrained=True)
    #vgg = self.change_actF(vgg)
    self.vgg = nn.Sequential(*list(vgg.children())[:-2])
    self.avgpool=nn.AdaptiveAvgPool2d(output_size=(2, 2))
    self.fc = nn.Sequential(
        nn.Linear(512*2*2, n_units),
        nn.GELU(),
        nn.Dropout(0.5),
        nn.Linear(n_units, n_units),
        nn.GELU(),
        nn.Dropout(0.5),
        nn.Linear(n_units, 10),
    )

  def forward(self,x):
    h = self.vgg(x)
    h = self.avgpool(h)
    h=h.reshape(h.size(0), -1)
    y = self.fc(h)
    return y

  def change_actF(self, net):
    Layers=list(net.children())[0]
    for i, layer in enumerate(Layers):
      if 'ReLU' in str(layer):
        net.features[i]=nn.GELU()
    return net

net_T=VGG()
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPUの定義
net_T = net_T.to(device)

print(net_T)
summary(net_T, input_size=(3,64,64))



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net_T.parameters(), lr=0.0001,  weight_decay=5e-4)

from torch.optim import lr_scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
num_epochs = 8 #学習エポックの設定

#グラフ作成のためにエポックごとの各値を保存する配列を作成
train_loss_list = []
train_acc_list = []


#学習の定義
for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    i=0#学習回数
    
    net_T.train() #train *1
    for (images, labels) in train_loader: #①
        images, labels = images.to(device), labels.to(device) #②
        optimizer.zero_grad() #③
        outputs = net_T(images) #④
        loss = criterion(outputs, labels)#⑤
        train_loss += loss.item()#⑥
        train_acc += (outputs.max(1)[1] == labels).sum().item() #⑦
        loss.backward()#⑧
        optimizer.step()#⑨
    scheduler.step()
        
        
    
    avg_train_loss = train_loss / len(train_loader.dataset) #⑩
    avg_train_acc = train_acc / len(train_loader.dataset)   #⑪
    
    print('epoch *', (epoch+1))
    print('Loss: {:.4f}, acc: {:.4f}%'.format(avg_train_loss, 100*avg_train_acc))
    #グラフ壁画用に各値を配列に格納
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)



事前学習なし

In [ ]:
from torchvision import models
class VGG(nn.Module):
  def __init__(self):
    super(VGG,self).__init__()
    n_units = 2048*2
    vgg = models.vgg19_bn(pretrained=False)
    #vgg = self.change_actF(vgg)
    self.vgg = nn.Sequential(*list(vgg.children())[:-2])
    self.avgpool=nn.AdaptiveAvgPool2d(output_size=(2, 2))
    self.fc = nn.Sequential(
        nn.Linear(512*2*2, n_units),
        nn.GELU(),
        nn.Dropout(0.5),
        nn.Linear(n_units, n_units),
        nn.GELU(),
        nn.Dropout(0.5),
        nn.Linear(n_units, 10),
    )

  def forward(self,x):
    h = self.vgg(x)
    h = self.avgpool(h)
    h=h.reshape(h.size(0), -1)
    y = self.fc(h)
    return y

  def change_actF(self, net):
    Layers=list(net.children())[0]
    for i, layer in enumerate(Layers):
      if 'ReLU' in str(layer):
        net.features[i]=nn.GELU()
    return net

net_F=VGG()
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPUの定義
net_F = net_F.to(device)

print(net_F)
summary(net_F, input_size=(3,64,64))



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net_F.parameters(), lr=0.0001,  weight_decay=5e-4)

from torch.optim import lr_scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
num_epochs = 8 #学習エポックの設定

#グラフ作成のためにエポックごとの各値を保存する配列を作成
train_loss_list = []
train_acc_list = []


#学習の定義
for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    i=0#学習回数
    
    net_F.train() #train *1
    for (images, labels) in train_loader: #①
        images, labels = images.to(device), labels.to(device) #②
        optimizer.zero_grad() #③
        outputs = net_F(images) #④
        loss = criterion(outputs, labels)#⑤
        train_loss += loss.item()#⑥
        train_acc += (outputs.max(1)[1] == labels).sum().item() #⑦
        loss.backward()#⑧
        optimizer.step()#⑨
    scheduler.step()
        
        
    
    avg_train_loss = train_loss / len(train_loader.dataset) #⑩
    avg_train_acc = train_acc / len(train_loader.dataset)   #⑪
    
    print('epoch *', (epoch+1))
    print('Loss: {:.4f}, acc: {:.4f}%'.format(avg_train_loss, 100*avg_train_acc))
    #グラフ壁画用に各値を配列に格納
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)



## 4. 学習記録のグラフ化
- matplotlibを用います。

In [ ]:
#学習のグラフ化
plt.figure()
plt.plot(range(num_epochs), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training accuracy')
plt.grid()

## 評価用セル(改変不可)

In [ ]:
score_T=eval(net_T)
print("事前学習ありのスコア",score_T)

score_F=eval(net_F)
print("事前学習なしのスコア",score_F)